# Run demanding OpenMM notebooks non-interactively on an HPC

**<span style="color:#A03;font-size:14pt">
&#x270B; HANDS-ON! &#x1F528;
</span>**

In the previous notebook, the simulation was performed making use of cpus. However, OpenMM (and molecular dynamics simulations in general)benefit of very large speed-ups when calculated on gpus. This notebook sets up the same simulation as the previous one to see this for yourself.
Note the small changes to the python code below and the sbatch script that now needs to be used: `job_openmm_using_gpu.sh`More specifically, the platform in the openmm simulation is changed to 'CUDA' below, while gpu resources are requested in the sbatch script.A comparison between a simulation using a full node on Victini and a single GPU on Accelgor leads to a speed-up of xxx for this simple example.

In [ ]:
%matplotlib widget

In [ ]:
# Import all the modules we need.
from sys import stdout

from openmm import *
from openmm.app import *
from openmm.unit import *

The following code was taken from [../02/02_alanine_dipeptide.ipynb](../02/02_alanine_dipeptide.ipynb), example 3.

In [ ]:
pdb = PDBFile("alanine-dipeptide.pdb")
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField("amber14-all.xml", "amber14/tip3pfb.xml")
modeller.addSolvent(forcefield, model="tip3p", padding=1 * nanometer)
print(modeller.topology)
# Write a PDB file to provide a topology of the solvated
# system to MDTraj below.
with open("init3.pdb", "w") as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

# The modeller builds a periodic box with the solute and solvent molecules.
# PME is the method to compute long-range electristatic interactions in
# periodic systems.
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
temperature = 300 * kelvin
pressure = 1 * bar
integrator = LangevinIntegrator(temperature, 1 / picosecond, 2 * femtoseconds)
system.addForce(MonteCarloBarostat(pressure, temperature))
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter("traj3.dcd", 100))
simulation.reporters.append(
    StateDataReporter(stdout, 1000, step=True, temperature=True, elapsedTime=True)
)
simulation.reporters.append(
    StateDataReporter(
        "scalars3.csv",
        100,
        time=True,
        potentialEnergy=True,
        totalEnergy=True,
        temperature=True,
    )
)
simulation.step(100000)